<a href="https://colab.research.google.com/github/PSuHyeon/Simple_TensorFlow/blob/main/Var_Fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Current state: 22 


TODO: package system





In [ ]:
import numpy as np
import weakref
import contextlib


In [ ]:
class Config:
  enable_backprop = None

@contextlib.contextmanager
def using_config(name, value):
  oldvalue = getattr(Config, name)
  setattr(Config, name, value)
  try:
    yield
  finally:
    setattr(Config, name, oldvalue)

def no_grad():
  return using_config('enable_backprop', False)

def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

def as_variable(obj):
  if isinstance(obj, Variable):
    return obj
  else:
    return Variable(obj)

Class Variable:

In [ ]:
class Variable:
  __array_priority_ = 200  #이건 ndarray 가 왼쪽에 올 때 Variable의 operator 가 우선권을 주기 위함임. 
  def __init__(self, data, name = None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = name
    self.grad = None
    self.creator = None
    self.generation = 0
  
  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim
  
  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  def set_creator(self, f):
    self.creator = f
    self.generation = f.generation + 1

  def cleargrad(self):
    self.grad = None

  def __len__(self):
    return len(self.data)
  
  def __repr__(self):
    if self.data is None: 
      return 'variable(None)'
    else:
      p = str(self.data).replace('\n', '\n' + ' ' * 9)
      return 'variable(' + p + ')'

  def backward(self, retain_grad = False):
    
    if self.grad is None: 
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key = lambda x: x.generation)

    add_func(self.creator)
    while funcs:
      f = funcs.pop()
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,) 
      for x, gx in zip(f.inputs, gxs): 
        if x.grad is not None:
          x.grad = x.grad + gx
        else:
          x.grad = gx
      
        if x.creator is not None:
          add_func(x.creator)
      if not retain_grad:
        for y in f.outputs:
          y().grad = None
  

Class Function (this will be overloaded into specific function):

  Define by run method --> as function proceed forward, result variable keeps tracks of creator (function/method) of which it was made. 

In [ ]:
class Function:

  def __call__(self, *inputs):
    inputs = [as_variable(input) for input in inputs]
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([input.generation for input in inputs])
      for o in outputs:
        o.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]

    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, x):
    raise NotImplementedError

  def backward(self, x):
    raise NotImplementedError

Specific higher functions:

In [ ]:
class Square(Function):

  def forward(self, input):
    y = input ** 2
    return y
  
  def backward(self, gy):
    gx = 2 * self.inputs[0].data * gy
    return gx

def square(x):
  return Square()(x)

In [ ]:
class Exp(Function):
  
  def forward(self, input):
    y = np.exp(input)
    return 
  
  def backward(self, gy):
    gx = np.exp(self.inputs[0].data) * gy
    return gx

def exp(x):
  return Exp()(x)

In [ ]:
class Add(Function):

  def forward(self, x0, x1):
    y = x0 + x1
    return y 

  def backward(self, gy):
    return gy, gy

def add(x1, x2):
  x1 = as_array(x1)
  return Add()(x1, x2)

Variable.__add__ = add
Variable.__radd__ = add

In [ ]:
class Mul(Function):
  
  def forward(self, x0, x1):
    y = x0 * x1
    return  y
  
  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    return x1 * gy, x0 * gy

def mul(x0, x1):
  x1 = as_array(x1)
  return Mul()(x0, x1)

Variable.__mul__ = mul
Variable.__rmul__ = mul

In [ ]:
class Neg(Function):
  def forward(self, x):
    return -x
  def backward(self, gy):
    return -gy
  
def neg(x):
  return Neg()(x)

Variable.__neg__ = neg 

In [ ]:
class Sub(Function):
  def forward(self, x0, x1):
    y = x0 - x1 
    return y
  def backward(self, gy):
    return gy, -gy 
  
def sub(x0, x1):
  x1 = as_array(x1)
  return Sub()(x0, x1)

def rsub(x0, x1): 
  x1 = as_array(x1)
  return Sub()(x1, x0)
Variable.__sub__ = sub
Variable.__rsub__ = rsub


In [ ]:
class Div(Function):
  def forward(self, x0, x1):
    y = x0 / x1
    return y
  
  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    gx0 = gy / x1
    gx1 = -gy * x0 / (x1 ** 2)
    return gx0, gx1

def div(x0, x1):
  x1 = as_array(x1)
  return Div()(x0, x1)

def rdiv(x0, x1):
  x1 = as_array(x1)
  return Div()(x1, x0)

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [ ]:
class Pow(Function):

  def __init__(self, c):
    self.c = c 

  def forward(self, x):
    c = self.c
    y = x ** c
    return y
  
  def backward(self, gy):
    x = self.inputs[0].data
    c = self.c
    gx = c * (x ** (c - 1)) * gy
    return gx

def pow(x,c):
  # c = as_array(c)
  return Pow(c)(x) 

Variable.__pow__ = pow


Numerical differentiation:

In [ ]:
def numerical_diff(f, x, e=1e-4):
  x0 = Variable(x.data + e)
  x1 = Variable(x.data - e)
  y0 = f(x0)
  y1 = f(x1)
  return (y0.data - y1.data) / (2 * e) 